# Vector boson fusion in IDM

The final notebook is [Significance_countours.ipynb](./Significance_countours.ipynb) which is based on [Sensitivity_fit.ipynb](Sensitivity_fit.ipynb) which is based on [Sensitivity_plots.ipynb](Sensitivity_plots.ipynb) by JD

### Initialization

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [13]:
import sys
import subprocess
import numpy as np
import pandas as pd
##pip3 install pyslha
#import pyslha # not longer required
import tempfile
import os
def subprocess_line_by_line(*args,TRUST_ERRORS=True,**kwargs):
    '''
    Subprocess output line by line. Stop of error found when TRUST_ERRORS=True, and simply
    report wait method otherwise.
    
    The arguments are the same as for the Popen constructor.
    
    WARNING: Works only in Python 3
    
    See: https://stackoverflow.com/a/28319191/2268280 
    and: https://stackoverflow.com/a/17698359/2268280
    
    Example:
    
    subprocess_line_by_line('for i in $(seq 1 3);do echo $i; sleep 1;done',shell=True)
    '''
    
    if not TRUST_ERRORS:
        kwargs['stderr']=subprocess.PIPE
        
    kwargs['stdout']=subprocess.PIPE
    kwargs['bufsize']=1
    kwargs['universal_newlines']=True
    s=subprocess.Popen(*args,**kwargs)
    with s as p:
        for line in p.stdout:
            print(line, end='') # process line here
    
    if TRUST_ERRORS:
        if p.returncode != 0:
            raise subprocess.CalledProcessError(p.returncode, p.args)
    else:
        return s.wait()

### Input variables

In [17]:
thisroot='/home/restrepo/prog/ROOT/root/bin/thisroot.sh'
#thisroot='/opt/root5/bin/thisroot.sh'
MHc=750
MH0=240
MADGRAPH='madgraph'
run_dir='Task_Asana'
work_dir='studies/IDM/'+run_dir
work_script='BP_'+str(int(MHc))+'_A_'+str(int(MH0))+'.txt'
LHA_input_file='MadGraph_cards/benchmarks/param_card_template.dat'

### Running flags

In [15]:
CLONE_GIT_REPO=False #WARNING: Try to overwrite current contents!
INSTALL=False # If True check full installation
TEST=False #take a long time
if not INSTALL:
    CLONE_GIT_REPO=False

## Install root 5

### Prerequisites 
```bash
apt-get install cmake git dpkg-dev make g++ gcc binutils libx11-dev libxpm-dev \
libxft-dev libxext-dev gfortran libssl-dev libpcre3-dev \
xlibmesa-glu-dev libglew1.5-dev libftgl-dev \
libmysqlclient-dev libfftw3-dev libcfitsio-dev \
graphviz-dev libavahi-compat-libdnssd-dev \
libldap2-dev python-dev libxml2-dev libkrb5-dev \
libgsl0-dev libqt4-dev r-base r-base-dev python-pip
```

Install ROOT 5 in some `PATH`
```bash
git clone http://root.cern.ch/git/root.git
cd root
git checkout v5-34-00-patches
./configure
make

```
Add to your `.bashrc`
```bash
source PATH/root/bin/thisroot.sh
```

In [18]:
f=open('kk.sh','w')
f.write('source '+thisroot+'\n')
f.write('which root\n')
f.close()

if not subprocess.Popen('bash kk.sh'.split(),
            stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()[0]:
    sys.exit('INSTALL ROOT: see instrucctions in notebook')

# Development of module to calculate one specific point

If `CLONE_GIT_REPO=False`, it is assumed that you did the clone as:
```bash
git clone  --recursive git@github.com:restrepo/VBF_IDM.git
```

In [166]:
if CLONE_GIT_REPO:  
    REPO='VBF_IDM'
    REPO_url='git@github.com:restrepo'
    work_dir='tmp'#'.' #WARNING: Try to overwirte contents
    if os.path.exists(work_dir+'index.ipynb'):
        sys.exit('ERROR: Repo files already exists. Check work_dir')
    if not os.path.isdir(work_dir):
        s=subprocess.Popen(['mkdir','-p',work_dir],stdout=subprocess.PIPE,stderr=subprocess.PIPE).wait()

    td=tempfile.mkdtemp()
    s=subprocess.Popen(('git clone  --recursive '+REPO_url+'/'+REPO+'.git').split(),cwd=td,
                 stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()

    s=subprocess.Popen('mv '+td+'/'+REPO+'/*  '+work_dir,shell=True,
                   stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()

    s=subprocess.Popen('mv '+td+'/'+REPO+'/.* '+work_dir,shell=True,
                   stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()

    os.rmdir(td+'/'+REPO)
    os.rmdir(td)

In [31]:
if INSTALL:
    s=subprocess.Popen('git checkout -b v2.3.3'.split(),cwd='madgraph',
                      stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()
    if 'Switched' not in s[1].decode('utf-8'):
        sys.exit('Submodule problems')

subprocess does not use .bashrc

In [19]:
f=open('madgraph/kk.sh','w')
f.write('source '+thisroot+'\n')
f.write('./bin/mg5_aMC install.dat\n')
f.close()

In [126]:
if INSTALL:
    s=subprocess.call('bash kk.sh'.split(),cwd='madgraph', stdout=open('kk','w'),stderr=open('kkk','w') )
    #subprocess_line_by_line('bash kk.sh'.split(),cwd='madgraph', TRUST_ERRORS=False )

In [ ]:
if TEST:
    s=subprocess.Popen('./test.sh'.split(), cwd='test',
            stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()

s[0].decode('utf-8')

### Create MadGraph script with processes

In [20]:
LambdasL=[0.01,0.02,0.05,0.07,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,1.0,3.0,5.0,7.0,10.0]

In [21]:
MH0s=np.array([70,75,80,85,90,110,130,150,170,190,210,220,240])

In [22]:
len(LambdasL)

20

```bash
rsiii@gfif:~/IDM_VBF/Samples/mh0_vs_lal/scan$ ls delphes_events_[0-9][0-9]_*.root | more
delphes_events_70_001_.root
delphes_events_70_002_.root
delphes_events_70_003_.root
...
...
delphes_events_90_018_.root
delphes_events_90_019_.root
delphes_events_90_020_.root
```

```bash
rsiii@gfif:~/IDM_VBF/Samples/mh0_vs_lal/scan$ ls delphes_events_[0-9][0-9][0-9]_*.root | more
delphes_events_110_001_.root
delphes_events_110_002_.root
delphes_events_110_003_.root
...
...
delphes_events_240_018_.root
delphes_events_240_019_.root
delphes_events_240_020_.root
```

cat readme.txt 
date 23/01/2017
Delphes output (.root) for the set the points contained in data_mh0_vs_x.dat

The data is organized as follows (in ascending order in lambda_L[lal] ):

#lal #mh0[GeV] #xs[fb]   #Delphes name
0.3 63.75 200        --> delphes_events_2.root
0.4 64.80 200        --> delphes_events_3.root 
0.5 66.32 200        --> delphes_events_4.root
0.6 68.05 200        --> delphes_events_5.root 
0.7 70.04 200        --> delphes_events_6.root
.   .     .          --> 
.   .     .          --> 
.   .     .          -->

The cross-section (xs) for the processes p p > h0 h0 j j
for all the set of points is around 200 fb. [Except for
the first point, with lal=0.3,  where the cross-section turns out to be 179 fb.]

In [23]:
df=pd.read_csv('Output_data.csv')

In [25]:
s=subprocess.Popen(['mkdir','-p',work_dir],
                 stdout=subprocess.PIPE, stderr=subprocess.PIPE).communicate()

In [26]:
def preamble(MHc=750,MH0=110):
    return '''import model InertDoublet_UFO
define p = g u c d s b u~ c~ d~ s~ b~
define j = p  
define l+ = e+ mu+ 
define l- = e- mu- 
define vl = ve vm vt 
define vl~ = ve~ vm~ vt~

generate p p > h2 h2 j j @0

output ../studies/IDM/BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lambdaL

'''

def lamL_loop(LHA_input_file,MHc=750,MH0=110,lamL=0.01):
    return '''launch ../studies/IDM/BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lambdaL
0    
../'''+LHA_input_file+'''
../Cards/run_card.dat
set wa0 auto
set whch auto 
set lamL '''+str(lamL)+'''
set mmh0 '''+str(MH0)+'''
set mma0 '''+str(MHc)+'''
set mmhch '''+str(MHc)+'''
0

''' 


def closing(MHc=750,MH0=110):
    return '''launch ../studies/IDM/BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lambdaL -i
print_results --path=./result_BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lamdaL_mh0_110.txt --format=short


done
'''

In [27]:
f=open(work_dir+'/'+work_script,'w')
f.write( preamble(MHc,MH0) )
for lamL in LambdasL:
    f.write( lamL_loop(LHA_input_file,MHc,MH0,lamL=lamL) )
f.write( closing(MHc,MH0) )
f.close()

```bash
# pip3 install subprocess.run && #pip2 install subprocess.run 
```

In [28]:
f=open(MADGRAPH+'/kk.sh','w')
f.write('source '+thisroot+'\n')
f.write('./bin/mg5_aMC ../'+work_dir+'/'+work_script+'\n')
f.close()

In [151]:
#s=subprocess.Popen( 'bash kk.sh'.split(), cwd=MADGRAPH,
#                                stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [ ]:
subprocess_line_by_line( 'bash kk.sh'.split(), cwd=MADGRAPH,TRUST_ERRORS=False)

************************************************************
*                                                          *
*                     W E L C O M E to                     *
*              M A D G R A P H 5 _ a M C @ N L O           *
*                                                          *
*                                                          *
*                 *                       *                *
*                   *        * *        *                  *
*                     * * * * 5 * * * *                    *
*                   *        * *        *                  *
*                 *                       *                *
*                                                          *
*         VERSION 2.3.3                 2015-10-25         *
*                                                          *
*    The MadGraph5_aMC@NLO Development Team - Find us at   *
*    https://server06.fynu.ucl.ac.be/projects/madgraph     *
*                       

INFO: Crossed process found for u c~ > h2 h2 b b~, reuse diagrams. 
INFO: Crossed process found for u d~ > h2 h2 u d~, reuse diagrams. 
INFO: Crossed process found for u d~ > h2 h2 u s~, reuse diagrams. 
INFO: Crossed process found for u d~ > h2 h2 u b~, reuse diagrams. 
INFO: Crossed process found for u d~ > h2 h2 c d~, reuse diagrams. 
INFO: Crossed process found for u d~ > h2 h2 c s~, reuse diagrams. 
INFO: Crossed process found for u d~ > h2 h2 c b~, reuse diagrams. 
INFO: Crossed process found for u s~ > h2 h2 u d~, reuse diagrams. 
INFO: Crossed process found for u s~ > h2 h2 u s~, reuse diagrams. 
INFO: Crossed process found for u s~ > h2 h2 u b~, reuse diagrams. 
INFO: Crossed process found for u s~ > h2 h2 c d~, reuse diagrams. 
INFO: Crossed process found for u s~ > h2 h2 c s~, reuse diagrams. 
INFO: Crossed process found for u s~ > h2 h2 c b~, reuse diagrams. 
INFO: Crossed process found for u b~ > h2 h2 u d~, reuse diagrams. 
INFO: Crossed process found for u b~ > h2 h2 u s

INFO: Crossed process found for d s~ > h2 h2 c c~, reuse diagrams. 
INFO: Crossed process found for d s~ > h2 h2 d s~, reuse diagrams. 
INFO: Crossed process found for d b~ > h2 h2 u u~, reuse diagrams. 
INFO: Crossed process found for d b~ > h2 h2 u c~, reuse diagrams. 
INFO: Crossed process found for d b~ > h2 h2 c u~, reuse diagrams. 
INFO: Crossed process found for d b~ > h2 h2 c c~, reuse diagrams. 
INFO: Crossed process found for d b~ > h2 h2 d b~, reuse diagrams. 
INFO: Process s g > h2 h2 g s added to mirror process g s > h2 h2 g s 
INFO: Process s u > h2 h2 u d added to mirror process u s > h2 h2 u d 
INFO: Process s u > h2 h2 u s added to mirror process u s > h2 h2 u s 
INFO: Process s u > h2 h2 u b added to mirror process u s > h2 h2 u b 
INFO: Process s u > h2 h2 c d added to mirror process u s > h2 h2 c d 
INFO: Process s u > h2 h2 c s added to mirror process u s > h2 h2 c s 
INFO: Process s u > h2 h2 c b added to mirror process u s > h2 h2 c b 
INFO: Process s c > h2 h2 u

INFO: Crossed process found for u~ d~ > h2 h2 u~ d~, reuse diagrams. 
INFO: Crossed process found for u~ d~ > h2 h2 u~ s~, reuse diagrams. 
INFO: Crossed process found for u~ d~ > h2 h2 u~ b~, reuse diagrams. 
INFO: Crossed process found for u~ d~ > h2 h2 c~ d~, reuse diagrams. 
INFO: Crossed process found for u~ d~ > h2 h2 c~ s~, reuse diagrams. 
INFO: Crossed process found for u~ d~ > h2 h2 c~ b~, reuse diagrams. 
INFO: Crossed process found for u~ s~ > h2 h2 u~ d~, reuse diagrams. 
INFO: Crossed process found for u~ s~ > h2 h2 u~ s~, reuse diagrams. 
INFO: Crossed process found for u~ s~ > h2 h2 u~ b~, reuse diagrams. 
INFO: Crossed process found for u~ s~ > h2 h2 c~ d~, reuse diagrams. 
INFO: Crossed process found for u~ s~ > h2 h2 c~ s~, reuse diagrams. 
INFO: Crossed process found for u~ s~ > h2 h2 c~ b~, reuse diagrams. 
INFO: Crossed process found for u~ b~ > h2 h2 u~ d~, reuse diagrams. 
INFO: Crossed process found for u~ b~ > h2 h2 u~ s~, reuse diagrams. 
INFO: Crossed proces

INFO: Crossed process found for s~ s~ > h2 h2 s~ s~, reuse diagrams. 
INFO: Crossed process found for s~ b~ > h2 h2 s~ b~, reuse diagrams. 
INFO: Process b~ g > h2 h2 g b~ added to mirror process g b~ > h2 h2 g b~ 
INFO: Process b~ u > h2 h2 u d~ added to mirror process u b~ > h2 h2 u d~ 
INFO: Process b~ u > h2 h2 u s~ added to mirror process u b~ > h2 h2 u s~ 
INFO: Process b~ u > h2 h2 u b~ added to mirror process u b~ > h2 h2 u b~ 
INFO: Process b~ u > h2 h2 c d~ added to mirror process u b~ > h2 h2 c d~ 
INFO: Process b~ u > h2 h2 c s~ added to mirror process u b~ > h2 h2 c s~ 
INFO: Process b~ u > h2 h2 c b~ added to mirror process u b~ > h2 h2 c b~ 
INFO: Process b~ c > h2 h2 u d~ added to mirror process c b~ > h2 h2 u d~ 
INFO: Process b~ c > h2 h2 u s~ added to mirror process c b~ > h2 h2 u s~ 
INFO: Process b~ c > h2 h2 u b~ added to mirror process c b~ > h2 h2 u b~ 
INFO: Process b~ c > h2 h2 c d~ added to mirror process c b~ > h2 h2 c d~ 
INFO: Process b~ c > h2 h2 c s~ add

...

INFO: Finding symmetric diagrams for subprocess group cs_h2h2cd 
INFO: Creating files in directory P0_cs_h2h2cb 
INFO: Generating Feynman diagrams for Process: c s > h2 h2 c b WEIGHTED=4 
INFO: Generating Feynman diagrams for Process: c s~ > h2 h2 c b~ WEIGHTED=4 
INFO: Generating Feynman diagrams for Process: c~ s~ > h2 h2 c~ b~ WEIGHTED=4 
INFO: Finding symmetric diagrams for subprocess group cs_h2h2cb 
INFO: Creating files in directory P0_cb_h2h2ud 
INFO: Generating Feynman diagrams for Process: c b > h2 h2 u d WEIGHTED=4 
INFO: Generating Feynman diagrams for Process: c b~ > h2 h2 u d~ WEIGHTED=4 
INFO: Generating Feynman diagrams for Process: c~ b~ > h2 h2 u~ d~ WEIGHTED=4 
INFO: Finding symmetric diagrams for subprocess group cb_h2h2ud 
INFO: Creating files in directory P0_cb_h2h2us 
INFO: Generating Feynman diagrams for Process: c b > h2 h2 u s WEIGHTED=4 
INFO: Generating Feynman diagrams for Process: c b~ > h2 h2 u s~ WEIGHTED=4 
INFO: Generating Feynman diagrams for Process: 

INFO: Finding symmetric diagrams for subprocess group dbx_h2h2ccx 
INFO: Creating files in directory P0_sux_h2h2dux 
INFO: Generating Feynman diagrams for Process: s u~ > h2 h2 d u~ WEIGHTED=4 
INFO: Finding symmetric diagrams for subprocess group sux_h2h2dux 
INFO: Creating files in directory P0_sux_h2h2dcx 
INFO: Generating Feynman diagrams for Process: s u~ > h2 h2 d c~ WEIGHTED=4 
INFO: Finding symmetric diagrams for subprocess group sux_h2h2dcx 
INFO: Creating files in directory P0_sux_h2h2scx 
INFO: Generating Feynman diagrams for Process: s u~ > h2 h2 s c~ WEIGHTED=4 
INFO: Finding symmetric diagrams for subprocess group sux_h2h2scx 
INFO: Creating files in directory P0_sux_h2h2bux 
INFO: Generating Feynman diagrams for Process: s u~ > h2 h2 b u~ WEIGHTED=4 
INFO: Finding symmetric diagrams for subprocess group sux_h2h2bux 
INFO: Creating files in directory P0_sux_h2h2bcx 
INFO: Generating Feynman diagrams for Process: s u~ > h2 h2 b c~ WEIGHTED=4 
INFO: Finding symmetric diagra

ALOHA: aloha creates SSS1 routines
ALOHA: aloha creates VVVS1 routines
ALOHA: aloha creates VVVVS3 routines
ALOHA: aloha creates VVVVS2 routines
ALOHA: aloha creates VVVVS1 routines
ALOHA: aloha creates FFV3 routines
ALOHA: aloha creates FFS1 routines
ALOHA: aloha creates VVS1 routines
ALOHA: aloha creates VVS2 routines
ALOHA: aloha creates VSS1 routines
save configuration file to /home/restrepo/prog/2017/VBF_IDM/studies/IDM/BP_750_240_vs_lambdaL/Cards/me5_configuration.txt
INFO: Use Fortran compiler gfortran 
INFO: Generate jpeg diagrams 
INFO: Generate web pages 
Output to directory /home/restrepo/prog/2017/VBF_IDM/studies/IDM/BP_750_240_vs_lambdaL done.
Type "launch" to generate events from this process, or see
/home/restrepo/prog/2017/VBF_IDM/studies/IDM/BP_750_240_vs_lambdaL/README
Run "open index.html" to see more information about this process.
launch ../studies/IDM/BP_750_240_vs_lambdaL
************************************************************
*                              

INFO:     P0_gg_h2h2uux  
INFO:     P0_gg_h2h2ccx  
INFO:     P0_gg_h2h2ddx  
INFO:     P0_gg_h2h2ssx  
INFO:     P0_gg_h2h2bbx  
INFO:     P0_gu_h2h2gu  
INFO:     P0_gc_h2h2gc  
INFO:     P0_gd_h2h2gd  
INFO:     P0_gs_h2h2gs  
INFO:     P0_gb_h2h2gb  
INFO:     P0_uux_h2h2gg  
INFO:     P0_ccx_h2h2gg  
INFO:     P0_ddx_h2h2gg  
INFO:     P0_ssx_h2h2gg  
INFO:     P0_bbx_h2h2gg  
INFO:     P0_ud_h2h2us  
INFO:     P0_ud_h2h2ub  
INFO:     P0_ud_h2h2cd  
INFO:     P0_ud_h2h2cs  
INFO:     P0_ud_h2h2cb  
INFO:     P0_us_h2h2ud  
INFO:     P0_us_h2h2ub  
INFO:     P0_us_h2h2cd  
INFO:     P0_us_h2h2cs  
INFO:     P0_us_h2h2cb  
INFO:     P0_ub_h2h2ud  
INFO:     P0_ub_h2h2us  
INFO:     P0_ub_h2h2cd  
INFO:     P0_ub_h2h2cs  
INFO:     P0_ub_h2h2cb  
INFO:     P0_uux_h2h2dsx  
INFO:     P0_uux_h2h2dbx  
INFO:     P0_uux_h2h2sdx  
INFO:     P0_uux_h2h2sbx  
INFO:     P0_uux_h2h2bdx  
INFO:     P0_uux_h2h2bsx  
INFO:     P0_ucx_h2h2ddx  
INFO:     P0_ucx_h2h2dsx  
INFO:     P0_ucx_h2h2dbx

INFO:  Idle: 1066,  Running: 8,  Completed: 356 [  6m 36s  ] 
INFO:  Idle: 1063,  Running: 8,  Completed: 359 [  6m 40s  ] 
INFO:  Idle: 1062,  Running: 8,  Completed: 360 [  6m 59s  ] 
INFO:  Idle: 1060,  Running: 8,  Completed: 362 [  7m 3s  ] 
INFO:  Idle: 1058,  Running: 8,  Completed: 364 [  7m 11s  ] 
INFO:  Idle: 1056,  Running: 8,  Completed: 366 [  7m 19s  ] 
INFO:  Idle: 1054,  Running: 8,  Completed: 368 [  7m 22s  ] 
INFO:  Idle: 1052,  Running: 8,  Completed: 370 [  7m 26s  ] 
INFO:  Idle: 1051,  Running: 8,  Completed: 371 [  7m 31s  ] 
INFO:  Idle: 1050,  Running: 8,  Completed: 372 [  7m 34s  ] 
INFO:  Idle: 1049,  Running: 8,  Completed: 373 [  7m 43s  ] 
INFO:  Idle: 1047,  Running: 8,  Completed: 375 [  7m 55s  ] 
INFO:  Idle: 1046,  Running: 8,  Completed: 376 [  7m 59s  ] 
INFO:  Idle: 1044,  Running: 8,  Completed: 378 [  8m 3s  ] 
INFO:  Idle: 1043,  Running: 8,  Completed: 379 [  8m 7s  ] 
INFO:  Idle: 1042,  Running: 8,  Completed: 380 [  8m 13s  ] 
INFO:  Idle

In [ ]:
s.wait()
#(MGOUT,MGERR)=s.communicate()

%%bash
cd madgraph/
./bin/mg5_aMC ../studies/IDM/Task_Asana/BP_750_A_110.txt
cd ..

### Prepare madevent script

In [22]:
pythia_script='TemplateRunPythiaDelphes_all.dat'
if len(LambdasL)>99:
    sys.exit('ERROR: UPDATE FORMAT FOR > 99 runs')
f=open(work_dir+'/'+pythia_script,'w')
for r in range(1,len(LambdasL)+1):
    f.write('pythia run_%02d\n' %r)
    f.write('3\n')
    f.write('0\n')
f.close()

In [23]:
output_dir='studies/IDM/BP_'+str(int(MHc))+'_'+str(int(MH0))+'_vs_lambdaL'

In [24]:
f=open(output_dir+'/kk.sh','w')
f.write('source '+thisroot+'\n')
f.write('./bin/madevent ../'+run_dir+'/'+pythia_script+'\n')
f.close()

In [18]:
s=subprocess.Popen('bash kk.sh'.split(), cwd=output_dir,
                                stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [19]:
s.wait()
(PHOUT,PHERR)=s.communicate()

print(PHOUT.decode('utf-8'))

### Get results

In [33]:
full_output='output'
s=subprocess.Popen(['mkdir', '-p',full_output]).wait()

#if 'xs_'+str(int(MH0)) not in df.columns.values:
if True:
    for r in range(1,len(LambdasL)+1):
        nrun='%02d' %r
        nrun3='%03d' %r
        s=subprocess.Popen(['cp',output_dir+'/Events/run_'+nrun+'/tag_1_delphes_events.root', 
                            full_output+'/delphes_events_'+str(int(MH0))+'_'+nrun3+'_.root'],
                             stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if s.wait()>0:
            sys.exit('Files not found')

ls studies/IDM/BP_750_110_vs_lambdaL/Events/run_07/